In [1]:
llm_config = {"model": "gpt-3.5-turbo"}

In [2]:
task = '''
    Write a concise but engaging blogpost 
    about the benefits of using a password manager.
    Make sure the blogpost is within 100 words. 
'''

# Create the writer agent 

In [3]:
import autogen 

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise"
    "blogpost (with title) on given topics. You must polish your writing"
    "based on the feedback you receive and give a refined version."
    "ONLY return your final work without additional comments.",
    llm_config=llm_config,
)

In [4]:
reply = writer.generate_reply(
    messages=[{"content": task, "role": "user"}]
)

In [5]:
print(reply)

Title: "Unlocking the Power of Password Managers"

In the digital age, safeguarding our online accounts is paramount. Enter password managers – the trusted sidekick in cybersecurity. These tools offer a range of benefits, from generating complex passwords to securely storing sensitive information. By using a password manager, you not only enhance security but also save time by auto-filling login details. Moreover, with the convenience of syncing across devices, you can access your passwords anytime, anywhere. Embrace the ease, enhance your security, and bid farewell to the hassle of remembering multiple passwords with a reliable password manager by your side.


# Adding reflection

In [6]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find ("TERMINATE") >= 0,
    system_message="You are a critic. You provide feedback on the writing"
    "style, grammar, and structure of the blogpost. You must provide"
    "constructive feedback to the writer in the aims of improving the quality of the content.",
    llm_config=llm_config,
)

In [7]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


    Write a concise but engaging blogpost 
    about the benefits of using a password manager.
    Make sure the blogpost is within 100 words. 


--------------------------------------------------------------------------------
Writer (to Critic):

Title: "Unlocking the Power of Password Managers"

In the digital age, safeguarding our online accounts is paramount. Enter password managers – the trusted sidekick in cybersecurity. These tools offer a range of benefits, from generating complex passwords to securely storing sensitive information. By using a password manager, you not only enhance security but also save time by auto-filling login details. Moreover, with the convenience of syncing across devices, you can access your passwords anytime, anywhere. Embrace the ease, enhance your security, and bid farewell to the hassle of remembering multiple passwords with a reliable password manager by your side.

-------------------------------------------------------------

# Nested Chat 

In [8]:
# Within the critic agent we can put in a bunch of other reviewers:

SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [9]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [10]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [11]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggregate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

In [12]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, 
    "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [13]:
# As soon as the critic receives the response from the writer, it will trigger the nested chats

critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

In [14]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


    Write a concise but engaging blogpost 
    about the benefits of using a password manager.
    Make sure the blogpost is within 100 words. 


--------------------------------------------------------------------------------
Writer (to Critic):

Title: "Unlocking the Power of Password Managers"

In the digital age, safeguarding our online accounts is paramount. Enter password managers – the trusted sidekick in cybersecurity. These tools offer a range of benefits, from generating complex passwords to securely storing sensitive information. By using a password manager, you not only enhance security but also save time by auto-filling login details. Moreover, with the convenience of syncing across devices, you can access your passwords anytime, anywhere. Embrace the ease, enhance your security, and bid farewell to the hassle of remembering multiple passwords with a reliable password manager by your side.

-------------------------------------------------------------

In [15]:
print(res.summary)

Title: "The Ultimate Guide to Harnessing Password Managers"

In today's digital era, fortifying your online presence is non-negotiable. Discover the game-changing advantages of employing a password manager. From generating robust passwords to securely storing sensitive data, these tools are indispensable in cybersecurity. Enhance your digital defense and save time with auto-fill capabilities. Seamlessly sync passwords across devices for on-the-go access. Say goodbye to password predicaments with a trusted password manager by your side. Unlock security, efficiency, and peace of mind with this powerhouse ally in the realm of cyberspace.
